# ES → BBF Salesforce Service_Charge Migration

This notebook migrates OrderItem records from ES Salesforce to Service_Charge__c in BBF Salesforce.

## Prerequisites
- **Service__c migration must be completed first** (Orders have BBF_New_Id__c populated)
- ES OrderItem records must have BBF_New_Id__c field created

## Object Mapping
- **ES Source:** OrderItem (Salesforce Standard Object with custom fields)
- **BBF Target:** Service_Charge__c (Custom Object)

## Process Overview
1. Connect to both ES (source) and BBF (target) Salesforce orgs
2. Query OrderItem from ES where:
   - Parent Order has BBF_New_Id__c populated (Service already migrated)
   - Parent Order Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
   - Parent Order Project_Group__c NOT LIKE '%PA MARKET DECOM%'
   - Parent Order Service_Order_Record_Type__c = 'Service Order Agreement'
   - OrderItem does NOT have BBF_New_Id__c populated (not yet migrated)
3. Transform ES OrderItem for BBF Service_Charge__c schema:
   - Map Order.BBF_New_Id__c → BBF Service ID (MASTER-DETAIL REQUIRED)
   - Map Product2 → Product_Simple__c picklist
   - Map charge type fields → Service_Type_Charge__c picklist
   - Set all Boolean fields to False (11 required fields)
   - Add ES_Legacy_ID__c = ES OrderItem.Id (for tracking)
4. Insert Service_Charge__c to BBF Salesforce
5. Update ES OrderItem with BBF_New_Id__c = BBF Service_Charge.Id
6. Create ID mapping: ES OrderItem ID → BBF Service_Charge ID
7. Output results to Excel with color-coded status

## Field Tracking Strategy
**In BBF Service_Charge__c:** `ES_Legacy_ID__c` stores original ES OrderItem ID
- Text(18), External ID, Unique (if available)

**In ES OrderItem:**
- `BBF_New_Id__c` stores new BBF Service_Charge ID after migration (Text 18)

## Safety
- `TEST_MODE = True` by default (limits to 10 Service Charges)
- Only migrates OrderItems where parent Order/Service is already migrated
- Skips OrderItems already migrated (BBF_New_Id__c populated)
- Skips OrderItems where parent Service not yet migrated
- Comprehensive error handling for master-detail relationship requirements

In [ ]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
import os

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Set-up successful")

In [ ]:
# === CONFIGURATION ===

# ES UAT Credentials
ES_USERNAME = "sfdcapi@everstream.net.uat"
ES_PASSWORD = "ZXasqw1234!@#$"
ES_TOKEN = "X0ation2CNmK5C0pV94M6vFYS"
ES_DOMAIN = "test"

# # ES (Source) Credentials - Production
# ES_USERNAME = "sfdcapi@everstream.net"
# ES_PASSWORD = "pV4CAxns8DQtJsBq!"
# ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
# ES_DOMAIN = "login"  # or 'test' for sandbox

# BBF (Target) Credentials
BBF_USERNAME = "vlettau@everstream.net"
BBF_PASSWORD = "MNlkpo0987)(*&"
BBF_TOKEN = "I4xmQLmm03cXl1O9qI2Z3XAAX"
BBF_DOMAIN = "test"  # or 'login' for production

# Migration Options
TEST_MODE = False  # ⚠️ Set to False to migrate ALL Service Charges
TEST_LIMIT = 10  # Only used when TEST_MODE = True

# 👤 Service Charge Owner - Set all migrated Service Charges to this user
OWNER_ID = (
    "005Ea00000ZOGFZIA5"  # Same as Account/Contact/BAN/Location/Service migration
)

# Output Configuration
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"es_bbf_service_charge_migration_{timestamp}.xlsx"

print("📋 Configuration loaded")
print(f"   TEST_MODE: {TEST_MODE}")
print(f"   Owner ID: {OWNER_ID}")
print(f"   Output: {output_file}")
print("\n⚠️  Note: Bulk API automatically handles batching (200 records/batch)")

In [ ]:
# === CONNECT TO SALESFORCE ORGS ===

print("=" * 80)
print("CONNECTING TO SALESFORCE ORGS")
print("=" * 80)

# Connect to ES (source)
print("\n📌 Connecting to ES (source)...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

# Connect to BBF (target)
print("\n📌 Connecting to BBF (target)...")
bbf_sf = Salesforce(
    username=BBF_USERNAME,
    password=BBF_PASSWORD,
    security_token=BBF_TOKEN,
    domain=BBF_DOMAIN,
)
print(f"✅ Connected to BBF: {bbf_sf.sf_instance}")

In [ ]:
# === QUERY ES ORDERITEM (Service Charges) ===
# Day 1: Only query fields we actually need
# BBF Service_Charge__c: Name is AUTONUMBER, no OwnerId
# Only need: Service__c (master-detail), Product_Simple__c, Service_Type_Charge__c, ES_Legacy_ID__c

print("\n" + "=" * 80)
print("QUERYING ES ORDERITEM (Service Charges) - MINIMAL FIELDS")
print("=" * 80)

# MINIMAL QUERY:
# - Id: for ES_Legacy_ID__c tracking
# - Order.BBF_New_Id__c: for master-detail to Service__c
# - Product2.Name: for Product_Simple__c picklist mapping
# - Charge_Type__c, MRC__c, NRC__c: for Service_Type_Charge__c mapping
query = """SELECT Id, Order.BBF_New_Id__c, Product2.Name, Charge_Type__c, MRC__c, NRC__c
FROM OrderItem
WHERE Order.BBF_New_Id__c != null
AND Order.BBF_New_Id__c != ''
AND Order.Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
AND (Order.Project_Group__c = null OR (NOT Order.Project_Group__c LIKE '%PA MARKET DECOM%'))
AND Order.Service_Order_Record_Type__c = 'Service Order Agreement'
AND (BBF_New_Id__c = null OR BBF_New_Id__c = '')"""

if TEST_MODE:
    query += f" LIMIT {TEST_LIMIT}"

print(f"Query:\n{query[:300]}...")
print("\nExecuting query...")

result = es_sf.query_all(query)
es_orderitems = result["records"]

print(f"✅ Found {len(es_orderitems)} OrderItem records to migrate")

if len(es_orderitems) > 0:
    sample = es_orderitems[0]
    print(f"\nSample OrderItem:")
    print(f"  ID:              {sample['Id']}")
    print(f"  BBF Service ID:  {sample.get('Order', {}).get('BBF_New_Id__c', 'N/A')}")
    print(f"  Product:         {sample.get('Product2', {}).get('Name', 'N/A')}")
else:
    print("\n✅ All OrderItem records have been migrated!")

In [ ]:
# === TRANSFORM FOR BBF SERVICE_CHARGE__c ===
# Day 1 Migration: REQUIRED FIELDS ONLY
# - Name: AUTONUMBER (don't set)
# - No OwnerId field on Service_Charge__c

print("\n" + "=" * 80)
print("TRANSFORMING ORDERITEM FOR BBF SERVICE_CHARGE__c (REQUIRED FIELDS ONLY)")
print("=" * 80)

bbf_service_charges = []
skipped_no_bbf_service = []


def map_product_simple(product_name):
    """Map ES Product to BBF Product_Simple__c picklist."""
    if not product_name:
        return "Other"
    product_upper = product_name.upper()
    if "ETHERNET" in product_upper or "E-LINE" in product_upper:
        return "Ethernet"
    elif "DARK FIBER" in product_upper or "DARK" in product_upper:
        return "Dark Fiber"
    elif "WAVELENGTH" in product_upper or "WAVE" in product_upper:
        return "Wavelength"
    elif "INTERNET" in product_upper or "DIA" in product_upper:
        return "Internet"
    elif "COLOCATION" in product_upper or "COLO" in product_upper:
        return "Colocation"
    elif "CLOUD" in product_upper:
        return "Cloud"
    elif "MANAGED" in product_upper:
        return "Managed Services"
    else:
        return "Other"


def map_service_type_charge(charge_type, mrc, nrc):
    """Map ES charge fields to BBF Service_Type_Charge__c picklist."""
    if charge_type:
        charge_upper = charge_type.upper()
        if "MRC" in charge_upper or "MONTHLY" in charge_upper or "RECURRING" in charge_upper:
            return "MRC"
        elif "NRC" in charge_upper or "ONE TIME" in charge_upper or "ONETIME" in charge_upper:
            return "NRC"
        elif "USAGE" in charge_upper:
            return "Usage"
    if mrc and float(mrc or 0) > 0:
        return "MRC"
    elif nrc and float(nrc or 0) > 0:
        return "NRC"
    return "MRC"


for es_orderitem in es_orderitems:
    bbf_service_id = None
    if es_orderitem.get("Order") and es_orderitem["Order"].get("BBF_New_Id__c"):
        bbf_service_id = es_orderitem["Order"]["BBF_New_Id__c"]

    if not bbf_service_id:
        skipped_no_bbf_service.append({
            "es_id": es_orderitem["Id"],
            "product": es_orderitem.get("Product2", {}).get("Name", "N/A"),
        })
        continue

    product_name = es_orderitem.get("Product2", {}).get("Name")
    product_simple = map_product_simple(product_name)
    service_type_charge = map_service_type_charge(
        es_orderitem.get("Charge_Type__c"),
        es_orderitem.get("MRC__c"),
        es_orderitem.get("NRC__c"),
    )

    # =========================================================================
    # BBF Service_Charge__c - REQUIRED FIELDS ONLY
    # Name = Autonumber (don't set)
    # No OwnerId on Service_Charge__c
    # =========================================================================
    bbf_service_charge = {
        # 🔴 REQUIRED: Master-Detail to Service__c
        "Service__c": bbf_service_id,
        # 🔴 REQUIRED: Picklists
        "Product_Simple__c": product_simple,
        "Service_Type_Charge__c": service_type_charge,
        # 🔗 Tracking
        "ES_Legacy_ID__c": es_orderitem["Id"],
    }

    bbf_service_charges.append(bbf_service_charge)

print(f"✅ Transformed {len(bbf_service_charges)} Service Charges")
print(f"\n   REQUIRED FIELDS ONLY:")
print(f"   - Service__c (Master-Detail)")
print(f"   - Product_Simple__c (mapped picklist)")
print(f"   - Service_Type_Charge__c (mapped picklist)")
print(f"   - ES_Legacy_ID__c (tracking)")
print(f"\n   Note: Name is Autonumber, no OwnerId on Service_Charge__c")

if len(skipped_no_bbf_service) > 0:
    print(f"\n⚠️  Skipped {len(skipped_no_bbf_service)} (no BBF Service ID)")

In [ ]:
# === INSERT TO BBF ===

print("\n" + "=" * 80)
print("INSERTING SERVICE CHARGES TO BBF")
print("=" * 80)

if len(bbf_service_charges) == 0:
    print("⚠️  No Service Charges to insert")
    successful_inserts = []
    failed_inserts = []
else:
    print(f"Inserting {len(bbf_service_charges)} Service Charges using bulk API...")
    print("(Bulk API automatically batches in 200-record chunks)\n")

    try:
        results = bbf_sf.bulk.Service_Charge__c.insert(bbf_service_charges)

        successful_inserts = []
        failed_inserts = []

        for i, result in enumerate(results):
            if result["success"]:
                successful_inserts.append(
                    {
                        "es_id": bbf_service_charges[i]["ES_Legacy_ID__c"],
                        "bbf_id": result["id"],
                        "name": bbf_service_charges[i].get("Name", "N/A"),
                        "bbf_service_id": bbf_service_charges[i]["Service__c"],
                        "service_type": bbf_service_charges[i].get(
                            "Service_Type_Charge__c", "N/A"
                        ),
                    }
                )
            else:
                failed_inserts.append(
                    {
                        "es_id": bbf_service_charges[i]["ES_Legacy_ID__c"],
                        "name": bbf_service_charges[i].get("Name", "N/A"),
                        "errors": result["errors"],
                        "bbf_service_id": bbf_service_charges[i]["Service__c"],
                    }
                )

        print(f"✅ Successfully inserted: {len(successful_inserts)} Service Charges")
        print(f"❌ Failed to insert: {len(failed_inserts)} Service Charges")

        if len(failed_inserts) > 0:
            print(f"\nFailed Service Charges (first 5):")
            for item in failed_inserts[:5]:
                print(f"  - {item['name']} (ES ID: {item['es_id']})")
                print(f"    Errors: {item['errors']}")

        if len(successful_inserts) > 0:
            print(f"\nSample successful insert:")
            sample = successful_inserts[0]
            print(f"  ES OrderItem ID:          {sample['es_id']}")
            print(f"  BBF Service_Charge ID:    {sample['bbf_id']}")
            print(f"  Name:                     {sample['name']}")
            print(f"  BBF Service ID:           {sample['bbf_service_id']}")
            print(f"  Service Type:             {sample['service_type']}")

    except Exception as e:
        print(f"❌ Error during bulk insert: {e}")
        successful_inserts = []
        failed_inserts = []

In [ ]:
# === UPDATE ES WITH BBF IDS ===

print("\n" + "=" * 80)
print("UPDATING ES WITH BBF IDS")
print("=" * 80)

if len(successful_inserts) == 0:
    print("⚠️  No Service Charges to update in ES")
    es_update_results = []
else:
    # Build update records for ES - set BBF_New_Id__c only
    es_updates = [
        {"Id": item["es_id"], "BBF_New_Id__c": item["bbf_id"]}
        for item in successful_inserts
    ]

    print(f"Updating {len(es_updates)} OrderItem records in ES...")
    print("   - Setting BBF_New_Id__c = BBF Service_Charge ID")

    try:
        es_update_results = es_sf.bulk.OrderItem.update(es_updates)

        success_count = sum(1 for r in es_update_results if r["success"])
        error_count = sum(1 for r in es_update_results if not r["success"])

        print(f"\n✅ Successfully updated: {success_count} OrderItem records in ES")
        print(f"❌ Failed to update: {error_count} OrderItem records in ES")

        if error_count > 0:
            print("\nFirst 10 update failures:")
            fail_count = 0
            for i, r in enumerate(es_update_results):
                if not r["success"] and fail_count < 10:
                    print(f"  - {es_updates[i]['Id']}: {r['errors']}")
                    fail_count += 1

    except Exception as e:
        print(f"❌ Error during ES update: {e}")
        es_update_results = []

In [ ]:
# === CREATE EXCEL OUTPUT ===

print("\n" + "=" * 80)
print("CREATING EXCEL OUTPUT")
print("=" * 80)

wb = Workbook()
ws1 = wb.active
ws1.title = "Migration Results"

# Styles
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_alignment = Alignment(horizontal="center", vertical="center")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

# Status colors
status_colors = {
    "Success": "C6EFCE",  # Green
    "Failed": "FFC7CE",  # Red
    "Skipped": "FFEB9C",  # Yellow
}

# --- SHEET 1: Migration Results ---
headers1 = [
    "ES OrderItem ID",
    "BBF Service_Charge ID",
    "Charge Name",
    "BBF Service ID",
    "Service Type",
    "Status",
    "Error",
]
ws1.append(headers1)

for col, header in enumerate(headers1, 1):
    cell = ws1.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment
    cell.border = thin_border

# Combine all results
all_results = []
for item in successful_inserts:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": item["bbf_id"],
            "Name": item["name"],
            "BBF_Service_ID": item["bbf_service_id"],
            "Service_Type": item["service_type"],
            "Status": "Success",
            "Error": "",
        }
    )
for item in failed_inserts:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "Name": item["name"],
            "BBF_Service_ID": item["bbf_service_id"],
            "Service_Type": "",
            "Status": "Failed",
            "Error": str(item["errors"]),
        }
    )
for item in skipped_no_bbf_service:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "Name": item["product"],
            "BBF_Service_ID": "",
            "Service_Type": "",
            "Status": "Skipped",
            "Error": "No BBF Service ID found (Master-Detail required)",
        }
    )

for row_idx, r in enumerate(all_results, 2):
    ws1.append(
        [
            r["ES_ID"],
            r["BBF_ID"],
            r["Name"],
            r["BBF_Service_ID"],
            r["Service_Type"],
            r["Status"],
            r["Error"],
        ]
    )
    fill_color = status_colors.get(r["Status"], "FFFFFF")
    for col in range(1, len(headers1) + 1):
        cell = ws1.cell(row=row_idx, column=col)
        cell.fill = PatternFill("solid", fgColor=fill_color)
        cell.border = thin_border

for col in ws1.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws1.column_dimensions[col[0].column_letter].width = min(max_length + 2, 60)

ws1.freeze_panes = "A2"

# --- SHEET 2: Summary ---
ws2 = wb.create_sheet("Summary")
ws2.append(["ES → BBF Service_Charge Migration Summary"])
ws2["A1"].font = Font(bold=True, size=14)
ws2.append([])
ws2.append(["Run Type:", "TEST MODE" if TEST_MODE else "FULL MIGRATION"])
ws2.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws2.append(["Owner ID:", OWNER_ID])
ws2.append([])
ws2.append(["Metric", "Count"])
ws2["A7"].font = Font(bold=True)
ws2["B7"].font = Font(bold=True)
ws2.append(["Total Service Charges Processed", len(all_results)])
ws2.append(["Successful Inserts", len(successful_inserts)])
ws2.append(["Failed Inserts", len(failed_inserts)])
ws2.append(["Skipped (No BBF Service)", len(skipped_no_bbf_service)])
ws2.append(
    [
        "Success Rate",
        (
            f"{len(successful_inserts)/len(all_results)*100:.1f}%"
            if len(all_results) > 0
            else "0%"
        ),
    ]
)

# --- SHEET 3: ID Mapping ---
ws3 = wb.create_sheet("ID Mapping")
headers3 = [
    "ES OrderItem ID",
    "BBF Service_Charge ID",
    "BBF Service ID",
    "Charge Name",
    "Service Type",
]
ws3.append(headers3)

for col, header in enumerate(headers3, 1):
    cell = ws3.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for item in successful_inserts:
    ws3.append(
        [
            item["es_id"],
            item["bbf_id"],
            item["bbf_service_id"],
            item["name"],
            item["service_type"],
        ]
    )

for col in ws3.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws3.column_dimensions[col[0].column_letter].width = min(max_length + 2, 50)

ws3.freeze_panes = "A2"

# --- SHEET 4: Failed Inserts ---
ws4 = wb.create_sheet("Failed Inserts")
headers4 = ["ES OrderItem ID", "Charge Name", "BBF Service ID", "Error Details"]
ws4.append(headers4)

for col, header in enumerate(headers4, 1):
    cell = ws4.cell(row=1, column=col)
    cell.font = Font(bold=True, size=12, color="FFFFFF")
    cell.fill = PatternFill(start_color="FF4444", end_color="FF4444", fill_type="solid")
    cell.alignment = header_alignment

for item in failed_inserts:
    ws4.append(
        [
            item["es_id"],
            item["name"],
            item["bbf_service_id"],
            str(item["errors"]),
        ]
    )

# Also add skipped records to failed inserts for tracking
for item in skipped_no_bbf_service:
    ws4.append(
        [
            item["es_id"],
            item["product"],
            "",
            "Skipped: No BBF Service ID found (Master-Detail required)",
        ]
    )

for col in ws4.columns:
    col_cells = list(col)
    max_length = max(len(str(cell.value)) for cell in col_cells) if col_cells else 10
    ws4.column_dimensions[col[0].column_letter].width = min(max_length + 2, 70)

ws4.freeze_panes = "A2"

# Save workbook
wb.save(output_file)
print(f"\n✅ Excel output saved to: {output_file}")
print(
    f"   📊 Sheet 1: Migration Results ({len(all_results)} Service Charges, color-coded)"
)
print(f"   📈 Sheet 2: Summary (metrics and stats)")
print(f"   🔗 Sheet 3: ID Mapping ({len(successful_inserts)} successful mappings)")
print(
    f"   ⚠️  Sheet 4: Failed Inserts ({len(failed_inserts) + len(skipped_no_bbf_service)} failures)"
)

In [ ]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("MIGRATION COMPLETE")
print("=" * 80)
print(f"ES OrderItem queried: {len(es_orderitems)}")
print(f"BBF Service_Charge__c inserted: {len(successful_inserts)}")
print(
    f"Success rate: {len(successful_inserts)/len(es_orderitems)*100:.1f}%"
    if len(es_orderitems) > 0
    else "N/A - No Service Charges processed"
)
print(f"\nExcel output: {output_file}")

if TEST_MODE:
    print(f"\n🔄 TEST MODE complete. Only migrated {TEST_LIMIT} Service Charges.")
    print(
        "   To migrate ALL Service Charges, set TEST_MODE = False in Cell 2 and re-run."
    )
else:
    print("\n✅ FULL MIGRATION complete!")
    print("   Service_Charge migration finished.")
    print("   Next: Migrate Off_Net__c (ES Off_Net__c → BBF Off_Net__c)")

if len(failed_inserts) > 0:
    print(f"\n⚠️  {len(failed_inserts)} Service Charges failed to insert")
    print("   Check 'Failed Inserts' sheet in Excel for details")

if len(skipped_no_bbf_service) > 0:
    print(
        f"\n⚠️  {len(skipped_no_bbf_service)} Service Charges skipped (no BBF Service - MASTER-DETAIL REQUIRED)"
    )
    print("   These OrderItems must have their parent Service migrated first")

---
## Next Steps: Off_Net__c Migration

After Service_Charge migration is complete, migrate Off_Net__c:

### Prerequisites for Off_Net__c Migration
| Prerequisite | Status | Notes |
|--------------|--------|-------|
| Account | ✅ Complete | Account migration done |
| BAN__c | ✅ Complete | BAN migration done |
| Contact | ✅ Complete | Contact migration done |
| Location__c | ✅ Complete | Location migration done |
| Service__c | ✅ Complete | Service migration done |
| Service_Charge__c | 🔄 This notebook | Use ID Mapping sheet |

### ID Mapping Files Needed for Off_Net__c
- `es_bbf_service_migration_*.xlsx` → Service ID mapping
- `es_bbf_location_migration_*.xlsx` → Location ID mapping

### ES Off_Net__c → BBF Off_Net__c
- Lookup: Off_Net__c.Service__c → Service__c.Id
- Lookup: Off_Net__c.Location__c → Location__c.Id
- Query: Off_Net__c WHERE Service__c has BBF_New_Id__c populated

## Field Mapping Reference

### ES OrderItem → BBF Service_Charge__c Field Mapping (Day 1 Minimum)

| ES Field | BBF Field | Notes |
|----------|-----------|-------|
| Id | ES_Legacy_ID__c | Tracking |
| Product2.Name | Name | Charge identifier |
| Order.BBF_New_Id__c | Service__c | MASTER-DETAIL (REQUIRED) |
| Product2 (mapped) | Product_Simple__c | Required picklist |
| Charge_Type__c (mapped) | Service_Type_Charge__c | Required picklist (MRC/NRC/etc) |
| (default False) | Active__c | Required boolean |
| (default False) | Charge_Active__c | Required boolean |
| (default False) | Disabled_COPS_Synchronization__c | Required boolean |
| (default False) | Ignore_in_MBB_Billing_Manually__c | Required boolean |
| (default False) | Ignore_in_MBB_Billing_Calculated__c | Required boolean |
| (default False) | Ignore_in_MBB_Billing_Backbill_Not_Found__c | Required boolean |
| (default False) | Ignore_in_MBB_Billing_Infra_Calc__c | Required boolean |
| (default False) | Do_Not_Backbill__c | Required boolean |
| (default False) | Exclude_From_Escalation__c | Required boolean |
| (default False) | Escalation_Created_Charge__c | Required boolean |
| (default False) | Price_Increase__c | Required boolean |
| Description | Description__c | Text |
| MRC__c | Amount__c | Currency (MRC) |
| NRC__c | NRC__c | Currency |
| UnitPrice | Unit_Rate__c | Currency |
| Quantity | Units__c | Number |
| ServiceDate | Start_Date__c | Date |
| EndDate | End_Date__c | Date |

## Product Mapping Logic

The notebook includes mapping logic for ES Product2 → BBF Product_Simple__c picklist:
- Ethernet products → 'Ethernet'
- Dark Fiber → 'Dark Fiber'
- Wavelength → 'Wavelength'
- Internet/DIA → 'Internet'
- Colocation → 'Colocation'
- Cloud → 'Cloud'
- Managed → 'Managed Services'
- Others → 'Other'

**Note:** Adjust the `map_product_simple()` function in Cell 5 based on actual BBF picklist values.

## Cleanup Apex (if needed)

### Delete Migrated Service Charges from BBF
```apex
List<Service_Charge__c> charges = [SELECT Id, Name FROM Service_Charge__c WHERE ES_Legacy_ID__c != null];
System.debug('Found ' + charges.size() + ' migrated Service Charges');
delete charges;
```

### Remove BBF_New_Id__c from ES (to re-run migration)
```apex
List<OrderItem> items = [SELECT Id, BBF_New_Id__c 
                         FROM OrderItem 
                         WHERE BBF_New_Id__c != NULL];
System.debug('Found ' + items.size() + ' records to reset');
for (OrderItem item : items) {
    item.BBF_New_Id__c = NULL;
}
update items;
```